In [15]:
!pip install openai langchain-community langchain-chroma langchain langchain-text-splitters langchain_core langchain_huggingface pypdf

In [1]:
from openai import OpenAI
import re
import math
from langchain_core.prompts import PromptTemplate

In [25]:
import json
from typing import List, Dict, Any
from dataclasses import dataclass
from enum import Enum

In [27]:
BASE_URL = "https://openrouter.ai/api/v1"
OPEN_API_KEY = "OPENROUTER_API_KEY"........................................YOUR_API_KEY
MODEL_1 = "mistralai/devstral-2512:free"
MODEL_2 = "xiaomi/mimo-v2-flash:free"

In [28]:
client=OpenAI(
    base_url=BASE_URL,
    api_key=OPEN_API_KEY
)

In [29]:
REACT_PROMPT = """
You are a ReAct agent.

You must follow this format exactly:

Thought: your reasoning
Action: tool_name[input]
Observation: result of the action
...
Thought: I now know the answer
Final Answer: answer to the user

Available tools:
- calculator
- search

User Question: {question}
"""


In [30]:
def prompt_making(user_input,keys, router_prompt):
    prompt = PromptTemplate(
        input_variables=["keys","user_input"],
        template=router_prompt
    )
    return prompt.format(input=user_input)

In [31]:


def response_generation(final_input):
    response_obj = client.responses.create(
        model=MODEL_1,
        input=final_input,
        max_output_tokens=1024,
        temperature=0.7,
        extra_headers={
            "HTTP-Referer": "https://company-chatbot.local",
            "X-Title": "Company RAG Chatbot",
        }
    )

    text = response_obj.output_text

    # Safe cleaning (do NOT break code formatting)
    clean_text = re.sub(r'[##_]+', '', text)

    return clean_text.strip()


In [32]:
class AgentPhase(Enum):
    """Different phases of agent reasoning"""
    UNDERSTANDING = "understanding"
    ANALYSIS = "analysis"
    PLANNING = "planning"
    EXECUTION = "execution"
    VERIFICATION = "verification"

@dataclass
class ReasoningStep:
    """Represents a single reasoning step"""
    phase: AgentPhase
    description: str
    findings: str
    confidence: float

@dataclass
class AgentPlan:
    """Agent's execution plan"""
    goal: str
    steps: List[str]
    reasoning: List[ReasoningStep]
    estimated_complexity: str


In [33]:
def search_tool(query: str):
    """Dummy search tool (replace with real search later)"""
    return f"Search result for '{query}': This is a placeholder result."


In [34]:
class DebuggingAgent:
    """AI Agent for code debugging with reasoning and planning"""

    def __init__(self, client):
        self.client = client
        self.reasoning_history = []
        self.current_plan = None

    def analyze_error(self, code: str, error_log: str) -> ReasoningStep:
        """Phase 1: Analyze the error and understand the problem"""
        prompt = f"""You are a debugging expert. Analyze this error carefully.

CODE:
{code}

ERROR LOG:
{error_log}

Identify:
1. The type of error
2. The root cause
3. Which part of the code is problematic

Be concise and specific."""

        analysis = response_generation(prompt)

        step = ReasoningStep(
            phase=AgentPhase.ANALYSIS,
            description="Error Analysis",
            findings=analysis,
            confidence=0.85
        )
        self.reasoning_history.append(step)
        return step

    def examine_configuration(self, code: str) -> ReasoningStep:
        """Phase 2: Check configuration and setup issues"""
        prompt = f"""Examine this code for configuration issues:

{code}

Look for:
1. Hardcoded credentials or secrets
2. Missing environment variables
3. Incorrect configuration patterns
4. Security vulnerabilities

List specific issues found."""

        findings = response_generation(prompt)

        step = ReasoningStep(
            phase=AgentPhase.ANALYSIS,
            description="Configuration Review",
            findings=findings,
            confidence=0.90
        )
        self.reasoning_history.append(step)
        return step

    def review_connection_logic(self, code: str) -> ReasoningStep:
        """Phase 3: Review database/connection logic"""
        prompt = f"""Review the connection logic in this code:

{code}

Check for:
1. Proper connection initialization
2. Error handling for connections
3. Connection pooling or management
4. Resource cleanup (closing connections)

Provide specific findings."""

        findings = response_generation(prompt)

        step = ReasoningStep(
            phase=AgentPhase.ANALYSIS,
            description="Connection Logic Review",
            findings=findings,
            confidence=0.88
        )
        self.reasoning_history.append(step)
        return step

    def create_plan(self, reasoning_steps: List[ReasoningStep]) -> AgentPlan:
        """Phase 4: Create a fix plan based on reasoning"""
        findings_summary = "\n\n".join([
            f"Step {i+1} ({step.description}):\n{step.findings}"
            for i, step in enumerate(reasoning_steps)
        ])

        prompt = f"""Based on these analysis findings, create a detailed fix plan:

{findings_summary}

Create a step-by-step plan to fix all identified issues. Be specific and actionable."""

        plan_text = response_generation(prompt)

        # Extract steps (simple parsing)
        steps = [line.strip() for line in plan_text.split('\n') if line.strip() and any(c.isdigit() for c in line[:3])]

        plan = AgentPlan(
            goal="Fix all identified bugs and improve code quality",
            steps=steps if steps else [plan_text],
            reasoning=reasoning_steps,
            estimated_complexity="Medium"
        )
        self.current_plan = plan
        return plan

    def generate_fix(self, code: str, plan: AgentPlan) -> str:
        """Phase 5: Generate the fixed code"""
        plan_summary = "\n".join([f"{i+1}. {step}" for i, step in enumerate(plan.steps)])

        prompt = f"""Generate fixed code based on this plan:

ORIGINAL CODE:
{code}

FIX PLAN:
{plan_summary}

REQUIREMENTS:
- Fix all identified issues
- Add proper error handling
- Use environment variables for credentials
- Add comments explaining key changes
- Ensure code follows best practices

Provide ONLY the complete fixed code, no explanations."""

        fixed_code = response_generation(prompt)
        return fixed_code

    def verify_fix(self, original_code: str, fixed_code: str) -> ReasoningStep:
        """Phase 6: Verify the fix addresses all issues"""
        prompt = f"""Compare the original and fixed code. Verify that all issues were addressed.

ORIGINAL:
{original_code[:500]}...

FIXED:
{fixed_code[:500]}...

List what was fixed and if anything was missed."""

        verification = response_generation(prompt)

        step = ReasoningStep(
            phase=AgentPhase.VERIFICATION,
            description="Fix Verification",
            findings=verification,
            confidence=0.92
        )
        self.reasoning_history.append(step)
        return step

    def debug_code(self, code: str, error_log: str) -> Dict[str, Any]:
        """Main method: Complete debugging workflow with reasoning and planning"""
        print("🤖 Starting AI Debugging Agent...")
        print("=" * 60)

        # Phase 1: Analyze Error
        print("\n📊 Phase 1: Analyzing Error...")
        error_analysis = self.analyze_error(code, error_log)
        print(f"✓ Confidence: {error_analysis.confidence:.0%}")
        print(f"Findings: {error_analysis.findings[:200]}...")

        # Phase 2: Check Configuration
        print("\n🔧 Phase 2: Checking Configuration...")
        config_review = self.examine_configuration(code)
        print(f"✓ Confidence: {config_review.confidence:.0%}")
        print(f"Findings: {config_review.findings[:200]}...")

        # Phase 3: Review Connection Logic
        print("\n🔌 Phase 3: Reviewing Connection Logic...")
        connection_review = self.review_connection_logic(code)
        print(f"✓ Confidence: {connection_review.confidence:.0%}")
        print(f"Findings: {connection_review.findings[:200]}...")

        # Phase 4: Create Plan
        print("\n📋 Phase 4: Creating Fix Plan...")
        plan = self.create_plan(self.reasoning_history)
        print(f"✓ Plan created with {len(plan.steps)} steps")
        for i, step in enumerate(plan.steps[:3], 1):
            print(f"  {i}. {step[:80]}...")

        # Phase 5: Generate Fix
        print("\n🔨 Phase 5: Generating Fixed Code...")
        fixed_code = self.generate_fix(code, plan)
        print(f"✓ Generated {len(fixed_code)} characters of fixed code")

        # Phase 6: Verify Fix
        print("\n✅ Phase 6: Verifying Fix...")
        verification = self.verify_fix(code, fixed_code)
        print(f"✓ Verification complete (Confidence: {verification.confidence:.0%})")

        print("\n" + "=" * 60)
        print("🎉 Debugging Complete!")

        return {
            "original_code": code,
            "fixed_code": fixed_code,
            "reasoning_history": self.reasoning_history,
            "plan": plan,
            "verification": verification
        }


In [36]:
BUGGY_CODE = """
const express = require('express');
const mysql = require('mysql');
const app = express();

// Database configuration
const dbConfig = {
  host: 'localhost',
  user: 'admin',
  password: 'wrong_password',
  database: 'myapp_db'
};

// Create connection
const connection = mysql.createConnection(dbConfig);

app.get('/users', (req, res) => {
  connection.query('SELECT * FROM users', (err, results) => {
    if (err) {
      res.status(500).send('Database error');
      return;
    }
    res.json(results);
  });
});

app.listen(3000, () => {
  console.log('Server running on port 3000');
});
"""

ERROR_LOG = """
Error: ER_ACCESS_DENIED_ERROR: Access denied for user 'admin'@'localhost' (using password: YES)
    at Protocol._enqueueCallback (/node_modules/mysql/lib/protocol/Protocol.js:171:48)
    at Protocol._parsePacket (/node_modules/mysql/lib/protocol/Protocol.js:449:23)

Timestamp: 2026-01-10 14:32:15
Status: Connection Failed
"""

In [38]:
if __name__ == "__main__":
    # Initialize agent
    agent = DebuggingAgent(client)  # Uncomment when client is ready

    # Run debugging process
    result = agent.debug_code(BUGGY_CODE, ERROR_LOG)

    # Display results
    print("\n" + "=" * 60)
    print("FIXED CODE:")
    print("=" * 60)
    print(result["fixed_code"])

    print("\n" + "=" * 60)
    print("REASONING SUMMARY:")
    print("=" * 60)
    for i, step in enumerate(result["reasoning_history"], 1):
        print(f"\nStep {i}: {step.description}")
        print(f"Phase: {step.phase.value}")
        print(f"Confidence: {step.confidence:.0%}")
        print(f"Findings: {step.findings[:150]}...")

🤖 Starting AI Debugging Agent...

📊 Phase 1: Analyzing Error...
✓ Confidence: 85%
Findings: 1. **Type of error**: Authentication error (`ERACCESSDENIEDERROR`)

2. **Root cause**: The provided MySQL credentials are incorrect. Specifically, the password `'wrongpassword'` is invalid for user `'...

🔧 Phase 2: Checking Configuration...
✓ Confidence: 90%
Findings: Here are the specific configuration issues found in the code:

1. **Hardcoded credentials**:
   - The database password `'wrongpassword'` is hardcoded directly in the configuration object
   - The use...

🔌 Phase 3: Reviewing Connection Logic...
✓ Confidence: 88%
Findings: Here's a review of the connection logic in the provided code:

 1. Proper connection initialization
- **Finding**: The connection is initialized correctly using `mysql.createConnection(dbConfig)`.
- *...

📋 Phase 4: Creating Fix Plan...
✓ Plan created with 13 steps
  1. 1. **Verify Correct MySQL Credentials**...
  2. 2. **Update `dbConfig` with Correct Credentia